# Data Pre-Processing

In [1]:
!pip install pytreebank

  Created wheel for pytreebank: filename=pytreebank-0.2.7-cp37-none-any.whl size=37070 sha256=9c2ede937c249cc31eaac14cc4567cf12db5ec1632d48b41343c7871d894b8ba
  Stored in directory: /root/.cache/pip/wheels/e0/b6/91/e9edcdbf464f623628d5c3aa9de28888c726e270b9a29f2368
Successfully built pytreebank


In [2]:
!pip install numpy git+https://github.com/makcedward/nlpaug.git

  Cloning https://github.com/makcedward/nlpaug.git to /tmp/pip-req-build-joexxkku
  Running command git clone -q https://github.com/makcedward/nlpaug.git /tmp/pip-req-build-joexxkku
  Created wheel for nlpaug: filename=nlpaug-1.1.3-cp37-none-any.whl size=837629 sha256=f68e848101e558e1ff9452d415163b95fa2b213f8ea8c5759f1f024fd3c8d880
  Stored in directory: /tmp/pip-ephem-wheel-cache-lwg9ug_j/wheels/2b/ef/30/a4e22f9a97373c9ab6763670c94aa5e111b0b956983f3892a4
Successfully built nlpaug


In [3]:
import pytreebank
import pandas as pd
import random
import nlpaug.augmenter.word as naw

In [4]:
dataset = pytreebank.load_sst()

In [5]:
[dataset.keys()]

[dict_keys(['train', 'test', 'dev'])]

In [6]:
len(dataset['train']), len(dataset['test']), len(dataset['dev'])

(8544, 2210, 1101)

In [7]:
train_df = pd.DataFrame({'sentence' : [dataset['train'][i].to_labeled_lines()[0][1] for i in range(len(dataset['train']))], 'labels' : [dataset['train'][i].to_labeled_lines()[0][0] for i in range(len(dataset['train']))]})
test_df = pd.DataFrame({'sentence' : [dataset['test'][i].to_labeled_lines()[0][1] for i in range(len(dataset['test']))], 'labels' : [dataset['test'][i].to_labeled_lines()[0][0] for i in range(len(dataset['test']))]})
val_df = pd.DataFrame({'sentence' : [dataset['dev'][i].to_labeled_lines()[0][1] for i in range(len(dataset['dev']))], 'labels' : [dataset['dev'][i].to_labeled_lines()[0][0] for i in range(len(dataset['dev']))]})
df = pd.concat([train_df, test_df, val_df]).reset_index(drop=True)
df = df.sample(frac=1).reset_index(drop=True)


In [8]:
df.shape

(11855, 2)

In [9]:
df.labels.value_counts()

1    3140
3    3111
2    2242
4    1852
0    1510
Name: labels, dtype: int64

In [10]:
df_train = df[:int(0.7*len(df))]
df_val = df[len(df_train):].reset_index(drop=True)

In [11]:
len(df_train), len(df_val)

(8298, 3557)

In [12]:
indexes = random.choices(list(range(len(df_train))), k = int(len(df_train)*0.05))                         
sentences = [df_train['sentence'][i] for i in indexes]
augmented_sentences = []

In [13]:
aug_del = naw.RandomWordAug(aug_p = 1)
aug_swap = naw.RandomWordAug(action="swap", aug_p = 1)
for i in range(len(sentences)//2):  
  augmented_text = aug_del.augment(sentences[i])  
  augmented_sentences += [augmented_text]
for i in range(len(sentences)//2, len(sentences)):  
  augmented_text = aug_swap.augment(sentences[i])  
  augmented_sentences += [augmented_text]

In [66]:
len(augmented_sentences)

414

In [14]:
df_train_aug = pd.DataFrame({'sentence': augmented_sentences, 'labels': [df_train['labels'][i] for i in indexes]})
df_aug = pd.concat([df_train, df_train_aug]).reset_index(drop=True)
df_aug = df_aug.sample(frac=1).reset_index(drop=True)

In [15]:
len(df_aug)

8712

# Model Utilities

In [16]:
import torch, torchtext
from torchtext import data

In [17]:
# Manual Seed
SEED = 43
torch.manual_seed(SEED)
CUDA_LAUNCH_BLOCKING=1

In [18]:
df_train = df_aug
df_val = df_val

In [19]:
df_val = df_val.reset_index(drop=True)

In [20]:
Sentence = torchtext.legacy.data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = torchtext.legacy.data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [21]:
fields = [('sentence', Sentence), ('label', Label)]

In [22]:
example_train = [torchtext.legacy.data.Example.fromlist([df_train.sentence[i],df_train.labels[i]], fields) for i in range(df_train.shape[0])] 
example_val = [torchtext.legacy.data.Example.fromlist([df_val.sentence[i],df_val.labels[i]], fields) for i in range(df_val.shape[0])] 



In [23]:
train = torchtext.legacy.data.Dataset(example_train, fields)
valid = torchtext.legacy.data.Dataset(example_val, fields)

In [25]:
len(train), len(valid)

(8712, 3557)

In [26]:
df_train.head()

,sentence,labels
0,Even these tales of just seven children seem a...,2
1,A sterling film - a cross between Boys Do n't ...,2
2,"Imagine Kevin Smith , the blasphemous bad boy ...",0
3,Imagine if you will a Tony Hawk skating video ...,1
4,Dignified CEO 's meet at a rustic retreat and ...,1


In [27]:
Sentence.build_vocab(train)
Label.build_vocab(train)

In [28]:
print('Size of input vocab : ', len(Sentence.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Sentence.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  17104
Size of label vocab :  5
Top 10 words appreared repeatedly : [('.', 8243), (',', 7347), ('the', 6137), ('and', 4451), ('of', 4398), ('a', 4360), ('to', 2985), ('-', 2885), ('is', 2477), ("'s", 2474)]
Labels :  defaultdict(None, {1: 0, 3: 1, 2: 2, 4: 3, 0: 4})


In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [30]:
train_iterator, valid_iterator = torchtext.legacy.data.BucketIterator.splits((train, valid), batch_size = 16, 
                                                            sort_key = lambda x: len(x.sentence),
                                                            sort_within_batch=True, device = device)

In [31]:
next(iter(train_iterator))
#len(train.examples[11].tweet)


[torchtext.legacy.data.batch.Batch of size 16]
	[.sentence]:('[torch.cuda.LongTensor of size 16x10 (GPU 0)]', '[torch.cuda.LongTensor of size 16 (GPU 0)]')
	[.label]:[torch.cuda.LongTensor of size 16 (GPU 0)]

In [42]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Sentence.vocab.stoi, tokens)

# Building Model and Prediction Process

In [85]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        #output = F.softmax(dense_outputs[0], dim=1)
            
        return dense_outputs[0]

In [86]:
# Define hyperparameters
size_of_vocab = len(Sentence.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 5
num_layers = 2
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [87]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(17104, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=5, bias=True)
)
The model has 5,373,305 trainable parameters


In [88]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [89]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        # resets the gradients after every batch
        optimizer.zero_grad() 
        
        
        # retrieve text and no. of words
        sentence, sentence_lengths = batch.sentence  
        
        # convert to 1D tensor
        predictions = model(sentence, sentence_lengths).squeeze()  
        #print(predictions, batch.label)
        # compute the loss
        loss = criterion(predictions, batch.label)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [90]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            sentence, sentence_lengths = batch.sentence
            
            # convert to 1d tensor
            predictions = model(sentence, sentence_lengths).squeeze() 
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [91]:
N_EPOCHS = 50
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 1.584 | Train Acc: 25.99%
	 Val. Loss: 1.572 |  Val. Acc: 26.83% 

	Train Loss: 1.573 | Train Acc: 26.67%
	 Val. Loss: 1.569 |  Val. Acc: 27.09% 

	Train Loss: 1.570 | Train Acc: 27.08%
	 Val. Loss: 1.568 |  Val. Acc: 26.15% 

	Train Loss: 1.568 | Train Acc: 27.35%
	 Val. Loss: 1.568 |  Val. Acc: 27.00% 

	Train Loss: 1.567 | Train Acc: 28.31%
	 Val. Loss: 1.567 |  Val. Acc: 26.95% 

	Train Loss: 1.565 | Train Acc: 28.70%
	 Val. Loss: 1.566 |  Val. Acc: 27.09% 

	Train Loss: 1.563 | Train Acc: 29.24%
	 Val. Loss: 1.566 |  Val. Acc: 27.12% 

	Train Loss: 1.561 | Train Acc: 29.70%
	 Val. Loss: 1.565 |  Val. Acc: 26.98% 

	Train Loss: 1.559 | Train Acc: 29.52%
	 Val. Loss: 1.564 |  Val. Acc: 27.23% 

	Train Loss: 1.557 | Train Acc: 30.29%
	 Val. Loss: 1.564 |  Val. Acc: 27.54% 

	Train Loss: 1.555 | Train Acc: 30.57%
	 Val. Loss: 1.563 |  Val. Acc: 27.06% 

	Train Loss: 1.553 | Train Acc: 30.94%
	 Val. Loss: 1.562 |  Val. Acc: 27.29% 

	Train Loss: 1.550 | Train Acc: 31.31%
	

In [92]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_sentence(sentence):
    
    categories = {0: "Very Negative", 1:"Negative", 2:"Neutral", 3:"Positive", 4:"Very Positive"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [93]:
classify_sentence("Jason X has cheesy effects and a hoary plot , but its macabre , self-deprecating sense of humor makes up for a lot .")

'Negative'